# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy

## The *Nim* and *Nimply* classes

In [2]:
# move definition, which row and number of object to remove
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        if k == None:
            self._k = self._rows[-1]
        else:
            self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    # action of removing
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, min(state.rows[row], state.k))
    return Nimply(row, num_objects)

In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, min(c + 1, state.k))]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [6]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


# compute for each possibile move the nim_sum and return a dictionary of possible_moves : { Numply: nim_sum , ... ,}
def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, min(c + 1, raw.k))):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")

    # take only the moves that have nim_sum != 0
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0 and ply.num_objects <= state.k]
    if not spicy_moves:
        spicy_moves = [move for move in list(analysis["possible_moves"].keys()) if move.num_objects <= state.k]
    ply = random.choice(spicy_moves)
    return ply

### Task 2.1
My implementation of Rule-Based agent based on Nim-Sum 

In [7]:
def expert_system(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")

    rows_not_zero = len(state.rows) - state.rows.count(0)

    # case of one single row with lenght >= 2
    if state.rows.count(1) == (rows_not_zero - 1):
        row, num_objects = [(row, num_objects) for row, num_objects in enumerate(state.rows) if num_objects > 1][0]
        if (rows_not_zero % 2) == 1:
            num_objects = num_objects if (num_objects - 1) <= state.k else state.k
            return Nimply(row, num_objects - 1)
        else:
            num_objects = num_objects if num_objects <= state.k else state.k
            return Nimply(row, num_objects)

    # case with more row with length >= 2
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    if not spicy_moves:
        spicy_moves = [move for move in list(analysis["possible_moves"].keys())]
    ply = random.choice(spicy_moves)
    return ply

## Oversimplified match

In [8]:
def match(nim, strategy):
    logging.getLogger().setLevel(logging.WARN)
    logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        logging.info(f"status: {nim}")
        player = 1 - player
    logging.info(f"status: Player {player} won!")
    return player

In [11]:
def test(nim, player_strategy, opponent_strategys, number_of_iteration):
    for strategy in opponent_strategys:
        wins = 0
        for _ in range(number_of_iteration):
            tmp = deepcopy(nim)
            winner = match(tmp, (player_strategy, strategy))
            if winner == 0:
                wins += 1
        print(
            f"Player 0 with strategy {player_strategy.__qualname__} vs Player 1 with strategy {strategy.__qualname__}: {(wins / number_of_iteration):.2%} win rate"
        )

In [12]:
nim = Nim(5)
opponent_strategys = (optimal, pure_random, gabriele)
test(nim, expert_system, opponent_strategys, 100)

Player 0 with strategy expert_system vs Player 1 with strategy optimal: 100.00% win rate
Player 0 with strategy expert_system vs Player 1 with strategy pure_random: 99.00% win rate
Player 0 with strategy expert_system vs Player 1 with strategy gabriele: 100.00% win rate
